# Insula API Processing Services

## Overview

This document serves as a quick reference guide of how to access Insula's APIs, using Python. This guide will show how to discover, launch and monitor processing services.


***Steps***:
1. Preparation
2. Discover a Processing Service
3. Launch a Processing Service

## Preparation

These preparation blocks have to be executed as a prerequisite for every subsequent step to work correctly as the imports will download every necessary package.
Insula uses the OIDC authentication, so check that the endpoints are correct and be sure to put a valid `USERNAME/TOKEN` pair.

In [ ]:
import requests
import base64
import json
import getpass

In [ ]:
USERNAME='service_developer'

In [ ]:
PASSWORD=getpass.getpass()

To authenticate correctly wWith `InsulaWorkflowClient` one must configure every endpoint correctly and is more generic.
The `InsulaOpenIDConnect` function will take care of providing or refreshing the token for the authentication every time `get_authorization_header()` is invoked.
This snippet can be run again if for any reason the token is expired.

In [ ]:
from InsulaWorkflowClient import InsulaOpenIDConnect

BASE_URL="https://biomass.pal.maap.eo.esa.int"

insulaAuth: InsulaOpenIDConnect = InsulaOpenIDConnect(
        authorization_endpoint="https://identity.pal.maap.eo.esa.int/realms/biomass/protocol/openid-connect/auth",
        token_endpoint="https://identity.pal.maap.eo.esa.int/realms/biomass/protocol/openid-connect/token",
        redirect_uri="http://localhost:9207/auth",
        client_id="api-client"
    )
insulaAuth.set_user_credentials(username=USERNAME, password=PASSWORD)

bearer = insulaAuth.get_authorization_header()
print(bearer)
HEADERS={'Authorization': bearer }

## Discover a Processing Service

This step is a straightforward one. By polling the `services` API we get a list of available services which can be used to execute a Processor in the next step.

In [ ]:
url=BASE_URL + "/secure/api/v2.0/services"
run_request=requests.get(url,headers=HEADERS)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

## Launch a Processing Service

### Create a Configuration

A Job Configuration is a config file that defines what the job should do. This request creates a new job config which is needed for the following section

The crucial aspects of a job config are:
- ***service***: the `service_id` instructs the job about which service is to be run
- ***inputs***: on which images/files said service is to be executed
- ***collection***: usually for many services this represents the output collection in which the results will be found

The processor must already be present, if it's not please follow the instructions inside `SimpleDemoProcessor` to create one that will work with the following snippet.
Be aware as these parameters must match with the ones that the service expects so this script will require some modification to work with a different processor.

In [ ]:
service_id=11 #JupyterSimpleDemoProcessor
collection_id="biomass3d15d7dd4d2c4ab9b581d002671d309b" #JupyterDemoOutput collection
s2="Insula is "
s3="very good!"

output={'result':collection_id}
job_config={
    'service': f"{BASE_URL}secure/api/v2.0/services/{service_id}",
    'inputs':{                                         #inputs must match the fields required by the service
        's2_input': [s2],
        's3_input': [s3],
        'collection': [json.dumps(output)]
    },
    'parent': None
}

url=BASE_URL+"/secure/api/v2.0/jobConfigs/"
run_request=requests.post(url,headers=HEADERS, data=json.dumps(job_config))
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

### Launch Processor
We can start a processor by getting the `job_config_id` from the previous response and launch it through the request you see in the following section:

In [ ]:
jobconfig_id=run_request_dict['id']
url=f"{BASE_URL}/secure/api/v2.0/jobConfigs/{jobconfig_id}/launch"
run_request = requests.post(url, headers=HEADERS)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

### Check Progress
To receive updates on the advancement of the job, we're simply getting the `job_id` from the previous response, and asking the paltform about it.

Feel free to run this section a couple of times to see the status changes until the job is completed.

In [ ]:
job_id=run_request_dict['id']
url_status = f"{BASE_URL}/secure/api/v2.0/jobs/{job_id}"
status = requests.get(url_status, headers=HEADERS)
url_status_dict = json.loads(status.text)

In [ ]:
url_status_dict